# Gene expression prediction (CNN version)
Reference: https://www.kaggle.com/c/gene-expression-prediction

Coded by Wangjin Lee
- jinsamdol@snu.ac.kr 

#### Description
Histone modifications are playing an important role in affecting gene regulation. Nowadays, predicting gene expression from histone modification signals is a widely studied research topic.

The dataset of this competition is on "E047" (Primary T CD8+ naive cells from peripheral blood) celltype from Roadmap Epigenomics Mapping Consortium (REMC) database. For each gene, it has 100 bins with five core histone modification marks [1]. (We divide the 10,000 basepair(bp) DNA region (+/-5000bp) around the transcription start site (TSS) of each gene into bins of length 100 bp [2], and then count the reads of 100 bp in each bin. Finally, the signal of each gene has a shape of 100x5.)

The goal of this competition is to develop algorithms for accurate predicting gene expression level. High gene expression level corresponds to target label = 1, and low gene expression corresponds to target label = 0.

Thus, the inputs are 100x5 matrices and target is the probability of gene activity.

#### Data description

*In this lesson, we will only use **training set**. The set consists of 'x_train.csv' and 'y_train.csv'.*

**x_train.csv**

This file contains the feature of the genes

A single gene (Field name: **GeneId**) has 500 reads information (shape: 100 x 5 ; 100 bp and 5 histones). The value in each cell for the histone denotes the number of reads in the read cluster.

The total number of the genes : 15,485

x_train.csv file looks like this ..

| GeneId | H3K4me3 | H3K4me1 | H3K36me3 | H3K9me3 | H3K27me3 |
|---|---|---|---|---|---|
| 1 | 2 | 1 | 4 | 1 | 0 |
| 1 | 0 | 2 | 1 | 1 | 1 |
| ... |
| 2  | 1 | 0 | 1 | 0 | 0 |
| ...  |
| 15485  | 0 | 0 | 0 | 2 | 1 |

The total number of the rows: 1,548,501 (15485 x 100 + 1)

The total number of the columns: 6 (1 + 5)

**x_train.csv**

This file contains the prediction whether a gene is expressed or not.

One prediction label for one GeneId.
Prediction label is binary (1: expressed, 0: suppressed)


| GeneId | Prediction |
|---|---|
| 1 | 0 |
| 2 | 0 |
| 3 | 1 |
| ... |
| 15485 | 0 |

The total number of the rows: 15,486 (15485 + 1)

The total number of the columns: 2 (1 + 1)

## Lesson: Curate a Dataset
The cells from here until **Loading the preprocessed data** include code for loading the data from the training files and storing them into variables, and dumping them into a preprocess file.

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd

#### Assign variable for each the data file path

In [ ]:
import os

data_path_x = os.path.abspath('./data/gene_expression/train/x_train.csv')
data_path_y = os.path.abspath('./data/gene_expression/train/y_train.csv')
print(data_path_x)

#### Read the x_train.csv file via pandas.read_csv( ) operation

In [ ]:
data_x = pd.read_csv(data_path_x, delimiter = ',', header=0)

#### Check the data description

In [ ]:
data_x.describe()

#### Check the some first rows in the data variable

In [ ]:
data_x = data_x.drop(columns=['GeneId'])
data_x.head()

#### Read the y_train.csv file via pandas.read_csv( ) operation

The prediction is on the second column (usecols=[1])

In [ ]:
data_y = pd.read_csv(data_path_y, delimiter = ',', header=0, usecols=[1])

#### Check the some first rows in the data variable

In [ ]:
data_y.head()

In [ ]:
data_y2 = data_y.values.tolist()

#### Check how the data_y2 looks like

In [ ]:
print(data_y2[0:10])

### Save the preprocessed data via pickle.dump( ) operation

In [ ]:
import pickle
with open('preprocess_ge_cnn.p', 'wb') as out_file:
    pickle.dump((data_x, data_y2), out_file)

## Lesson: Load the dataset

#### After storing the preprocessed data into a file, we can begin at here and save our time.

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd

In [2]:
import pickle
def load_preprocess():
    with open('preprocess_ge_cnn.p', mode='rb') as in_file:
        return pickle.load(in_file)
(all_data_x, all_data_y) = load_preprocess()

### We need to scale (normalize) the x_data. 
Through the output from the data_x.describe() above, we checked that the data in each cell has large variance. It may lead poor prediction performance in neural network.

In [3]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_x = scaler.fit_transform(all_data_x).tolist()


In [4]:
print(len(all_data_x), len(all_data_y), len(scaled_x))
print(scaled_x[0:2])

1548500 15485 1548500
[[0.17557490236948337, -0.2932485067426945, 0.13486841189835097, -0.3322358747505786, -0.7980722505108142], [-0.7690025785016734, 0.2563978841954451, -0.5056471723258046, -0.3322358747505786, -0.08486416286080267]]


### data to CNN input style

In [5]:
data_cnn_x = []
bp = 100
for i in range(len(all_data_y)):
    #a = np.reshape(scaled_x[i*bp : (i+1)*bp], (100,5))
    data_cnn_x.append(np.reshape(scaled_x[i*bp : (i+1)*bp], (100,5,1)).tolist())

In [6]:
print(np.shape(data_cnn_x))

(15485, 100, 5, 1)


### One-Hot Encoding
The label in a binary or n-ary classification problem can be encoded via One-Hot Encoding method.
One-Hot encoding for binary classification problem can be seen as a method encoding label in 2-dimensional vector. 

Example: 

|Prediction |
|---|
|0|
|1|
|0|
|0|

- 1-dimensional vector: [[0], [1], [0], [0]]

- One-hot encoding (2-dimensional vector): [[1, 0], [0, 1], [1, 0], [1, 0]]

Each dimension in the 2-d vector denotes each element, thus, the *n* is larger, more zeros in the encoding vector.

In [7]:
#all_data_y_list = all_data_y.tolist()
print(all_data_y[0:5])

[[0], [0], [1], [1], [1]]


In [8]:
from sklearn.preprocessing import OneHotEncoder
data_y = OneHotEncoder(sparse=False).fit_transform(all_data_y).tolist()
print(data_y[0:5])

[[1.0, 0.0], [1.0, 0.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0]]


### Separation of the data → Training / Development / Testing

#### Check the X-Y pair and the number of instances

In [9]:
n_total = len(data_cnn_x)
print(n_total)
n_total_y = len(data_y)
assert n_total == n_total_y

15485


#### Warning!!: In the case you want to use small pieces of the data set, run this cell below, or, skip this cell

In [ ]:
'''
Using only 1/n of the whole data
'''
import math
_n=3
data_cnn_x = data_cnn_x[:math.floor(n_total/_n)]
data_y = data_y[:math.floor(n_total/_n)]

n_total = len(data_cnn_x)
print(n_total)
n_total_y = len(data_y)
assert n_total == n_total_y

We will divide the whole set into 7:1:2 for training, dev, and test, respectively.

In [10]:
x_data_train = data_cnn_x[:n_total//10*7]
y_data_train = data_y[:n_total//10*7]

x_data_dev = data_cnn_x[n_total//10*7:n_total//10*8]
y_data_dev = data_y[n_total//10*7:n_total//10*8]

x_data_test = data_cnn_x[n_total//10*8:]
y_data_test = data_y[n_total//10*8:]
print ('#train: ',' ', len(x_data_train))
print ('#dev: ',' ', len(x_data_dev))
print ('#test: ',' ', len(x_data_test))

#train:    10836
#dev:    1548
#test:    3101


In [11]:
np.shape(x_data_train)

(10836, 100, 5, 1)

## Lesson: Build the graph

In [12]:
#constants
x_col = 5
x_row = 100
x_cell = x_row*x_col
n_classes=2

#### Hyperparameters

* We will design a neural network graph consisting of one input layer, three hidden layers, and one output layer.
* In the cell below, set your hyper parameters for the node number for each layer
* Input layer (size 500) → Layer 1 (size ?) → Activation → Dropout → Layer 2 (size ?) → Activation → Dropout → Layer 3 (size ?) → Activation → Output layer (size ?) → Softmax
* Also, we can control the learning rate and *keep probability* in the dropout layer. Set your hyperparameters for the values.

In [13]:
lr=0.001 #learning rate
kp=0.4 #keep probability (dropout layer)
n_iter=5000

### This is the graph structure in the TF code. 
You may use this structure and can modify the structure.
* tf.name_scope('name') will make a group in the graph. It is very useful tip making your graph compact when you see your graph in the tensorboard.
* If you want to see scalar (cost), use **tf.summary.scalar('name', variable)**
* If you want to see histogram (weights, bias), use **tf.summary.histogram('name', variable)**
* Before running the training session, type **your_summary_name=tf.summary.merge_all()**


In [29]:
tf.reset_default_graph()

def conv2d_maxpool(x_tensor, conv_num_outputs, conv_ksize, conv_strides, pool_ksize, pool_strides):
    """
    Apply convolution then max pooling to x_tensor
    :param x_tensor: TensorFlow Tensor
    :param conv_num_outputs: Number of outputs for the convolutional layer
    :param conv_ksize: kernal size 2-D Tuple for the convolutional layer
    :param conv_strides: Stride 2-D Tuple for convolution
    :param pool_ksize: kernal size 2-D Tuple for pool
    :param pool_strides: Stride 2-D Tuple for pool
    : return: A tensor that represents convolution and max pooling of x_tensor
    """
    # TODO: Implement Function
    
    #print('x_tensor.shape: ', x_tensor.shape)
    n_input_feature_map = x_tensor.get_shape()[3].value
    #print('feature_map: ', n_input_feature_map)
    
    #print('conv_num_outputs: ', conv_num_outputs)
    #print('conv_ksize: ', conv_ksize)
    #print('conv_strides: ', conv_strides)
    #print('pool_ksize: ', pool_ksize)
    #print('pool_strides: ', pool_strides)
    
    #Create the weight and bias using conv_ksize, conv_num_outputs and the shape of x_tensor    
    W = tf.Variable(tf.truncated_normal([conv_ksize[0], conv_ksize[1], n_input_feature_map, conv_num_outputs], mean = 0.0, stddev=0.05),\
                    name='weight')
    #print('shape_of_the_weight: ', W.shape)
    
    b = tf.Variable(tf.zeros(conv_num_outputs), dtype=tf.float32, name='bias')
    #print('shape_of_the_bias: ', b.shape)
    
    #apply a convolution to x_tensor using weight and conv_strides (with same padding)
    convolution = tf.nn.conv2d(x_tensor, W, strides=[1, conv_strides[0], conv_strides[1], 1], padding='SAME')        
    convolution = tf.nn.bias_add(convolution, b)#add bias        
    convolution = tf.nn.relu(convolution)#add a nonlinear activation to the convolution
    
    #apply max pooling using pool_ksize and pool_strides
    convolution = tf.nn.max_pool(convolution, ksize=[1, pool_ksize[0], pool_ksize[1], 1],\
                                 strides=[1, pool_strides[0], pool_strides[1], 1], padding='SAME')
    
    return convolution




In [30]:
def flatten(x_tensor):
    """
    Flatten x_tensor to (Batch Size, Flattened Image Size)
    : x_tensor: A tensor of size (Batch Size, ...), where ... are the image dimensions.
    : return: A tensor of size (Batch Size, Flattened Image Size).
    """
    # TODO: Implement Function
    #print(x_tensor.shape)
    
    W = x_tensor.get_shape()[1].value    
    H = x_tensor.get_shape()[2].value    
    D = x_tensor.get_shape()[3].value
    
    #print ('W:', W, ' H:', H, ' D:', D, ' #:', W*H*D)
    
    flatten = tf.reshape(x_tensor, [-1, W*H*D])
    
    
    #print(flatten.shape)
    return flatten


In [31]:
def fully_conn(x_tensor, num_outputs):
    """
    Apply a fully connected layer to x_tensor using weight and bias
    : x_tensor: A 2-D tensor where the first dimension is batch size.
    : num_outputs: The number of output that the new tensor should be.
    : return: A 2-D tensor where the second dimension is num_outputs.
    """
    # TODO: Implement Function
    #print(x_tensor.get_shape()[1].value)
    W = tf.Variable(tf.truncated_normal( [x_tensor.get_shape()[1].value, num_outputs] , mean = 0.0, stddev=0.05),  name='weight')
    b = tf.Variable(tf.zeros(num_outputs), dtype=tf.float32, name='bias')
    
    fc = tf.add(tf.matmul(x_tensor, W), b)
    fc = tf.nn.relu(fc)
    return fc

In [32]:
def output(x_tensor, num_outputs):
    """
    Apply an output layer to x_tensor using weight and bias
    : x_tensor: A 2-D tensor where the first dimension is batch size.
    : num_outputs: The number of output that the new tensor should be.
    : return: A 2-D tensor where the second dimension is num_outputs.
    """
    # TODO: Implement Function
    #print(x_tensor.get_shape()[1].value)
    W = tf.Variable(tf.truncated_normal([x_tensor.get_shape()[1].value, num_outputs], mean = 0.0, stddev=0.05), name='weight')
    b = tf.Variable(tf.zeros(num_outputs), dtype=tf.float32, name='bias')
    
    out = tf.add(tf.matmul(x_tensor, W), b)
    
    #without softmax
    
    return out

In [33]:
def conv_net(x, keep_prob):
    """
    Create a convolutional neural network model
    : x: Placeholder tensor that holds image data.
    : keep_prob: Placeholder tensor that hold dropout keep probability.
    : return: Tensor that represents logits
    """
    # TODO: Apply 1, 2, or 3 Convolution and Max Pool layers
    #    Play around with different number of outputs, kernel size and stride
    # Function Definition from Above:
    #    conv2d_maxpool(x_tensor, conv_num_outputs, conv_ksize, conv_strides, pool_ksize, pool_strides)

    num_classes = 2
    
    #layer 1
    conv_num_outputs1 = 50
    conv_ksize1 = (3, 3)
    conv_strides1 = (1, 1)
    pool_ksize1 = (2, 2)
    pool_strides1 = (2, 2)
           
    conv_layer1 = conv2d_maxpool(x, conv_num_outputs1, conv_ksize1, conv_strides1, pool_ksize1, pool_strides1)
    conv_layer1 = tf.nn.dropout(conv_layer1, keep_prob)
    
    #layer 2
    conv_num_outputs2 = 24
    conv_ksize2 = (3, 3)
    conv_strides2 = (1, 1)
    pool_ksize2 = (2, 2)
    pool_strides2 = (2, 2)
    
    conv_layer2 = conv2d_maxpool(conv_layer1, conv_num_outputs2, conv_ksize2, conv_strides2, pool_ksize2, pool_strides2)
    
    
    # TODO: Apply a Flatten Layer
    # Function Definition from Above:
    #   flatten(x_tensor)
    flat_layer = flatten(conv_layer2)
    

    # TODO: Apply 1, 2, or 3 Fully Connected Layers
    #    Play around with different number of outputs
    # Function Definition from Above:
    #   fully_conn(x_tensor, num_outputs)
    
    fc = fully_conn(flat_layer, 12)
    fc = tf.nn.dropout(fc, keep_prob=keep_prob)
    
    # TODO: Apply an Output Layer
    #    Set this to the number of classes
    # Function Definition from Above:
    #   output(x_tensor, num_outputs)
    
    out = output(fc, num_classes)
    
    
    
    # TODO: return output
    return out

   #layer 1
    conv_num_outputs1 = 30
    conv_ksize1 = (3, 3)
    conv_strides1 = (1, 1)
    pool_ksize1 = (2, 2)
    pool_strides1 = (2, 2)
           
    #layer 2
    conv_num_outputs2 = 10
    conv_ksize2 = (3, 3)
    conv_strides2 = (1, 1)
    pool_ksize2 = (2, 2)
    pool_strides2 = (1, 1)
    
    flat_layer = flatten(conv_layer2)
    

    
    fc = fully_conn(flat_layer, 5)
    fc = tf.nn.dropout(fc, keep_prob=keep_prob)

    #layer 1
    conv_num_outputs1 = 10
    conv_ksize1 = (1, 3)
    conv_strides1 = (1, 1)
    pool_ksize1 = (1, 1)
    pool_strides1 = (1, 1)
           
   
    #layer 2
    conv_num_outputs2 = 5
    conv_ksize2 = (1, 3)
    conv_strides2 = (1, 1)
    pool_ksize2 = (1, 1)
    pool_strides2 = (1, 1)
    
    conv_layer2 = conv2d_maxpool(conv_layer1, conv_num_outputs2, conv_ksize2, conv_strides2, pool_ksize2, pool_strides2)
    
   
    flat_layer = flatten(conv_layer2)
    
    fc = fully_conn(flat_layer, 3)
    fc = tf.nn.dropout(fc, keep_prob=keep_prob)
    out = output(fc, num_classes)

In [34]:
'''
double layers
'''

ncol = len(scaled_x[0])
nclass = n_classes

with tf.name_scope('inputs'):
    
    #X = tf.placeholder(tf.float32, shape=[None, x_cell], name='x')
    #X_img = tf.reshape(X, shape=[None, x_col, x_row, 1], name='x_image')
    X = tf.placeholder(tf.float32, shape=(None, x_row, x_col, 1), name='x')
    Y = tf.placeholder(tf.float32, shape=[None, nclass], name='y')


    
with tf.name_scope('conv'):
    # Model
    logits = conv_net(X, kp)

    # Name logits Tensor, so that is can be loaded from disk after training
    logits = tf.identity(logits, name='logits')

with tf.name_scope('trainer'):
    
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels = Y))    
    optimizer = tf.train.AdamOptimizer(learning_rate=lr).minimize(cost)        
    
with tf.name_scope('predictor'):
    
    correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
    accuracy = tf.reduce_mean (tf.cast(correct_pred, tf.float32))
    
    

tf.summary.scalar('cost', cost)
tf.summary.scalar('accuracy', accuracy)

merged = tf.summary.merge_all()


## Training and testing your graph.

* If you want to save tensorboard log file, type like this just after sess.run(tf.global_variables_initializer())

  **file_name = tf.summary.FileWriter('log_folder_name', sess.graph)**

* If your want to save the variables during the training, run the code

  **summary = sess.run([your_summary_name], feed_dict={....})**
  **file_name.add_summary(summary, step)**

In [35]:

sess = tf.Session()
with sess.as_default():
    
    sess.run(tf.global_variables_initializer())

    #for 1 layer
    fw2 = tf.summary.FileWriter('./tb/gene_expression_cnn/train/model{}'.format(3), sess.graph)
    fw_val = tf.summary.FileWriter('./tb/gene_expression_cnn/validation/model{}'.format(3))
    fw_test = tf.summary.FileWriter('./tb/gene_expression_cnn/test/model{}'.format(3))
    

    for step in range(n_iter+1):

        
        summary, _, c = sess.run([merged, optimizer, cost], feed_dict={X:x_data_train, Y:y_data_train})
        
        if step % 200 == 0:
            summary_v, acc = sess.run([merged, accuracy], feed_dict={X:x_data_dev, Y:y_data_dev})
            print(step, '\tcost:', c, '\tacc:', acc)
            #print(summary_v)
            fw_val.add_summary(summary_v, step)
            
        fw2.add_summary(summary, step)
        
        
        
    summary_t, acc_t = sess.run([merged, accuracy], feed_dict={X:x_data_test, Y:y_data_test})    
    fw_test.add_summary(summary_t, step)
    print('final accuracy: ', acc)
    

0 	cost: 0.6988085 	acc: 0.48901808
200 	cost: 0.44161975 	acc: 0.7209302
400 	cost: 0.41314155 	acc: 0.7661499
600 	cost: 0.38850668 	acc: 0.78488374
800 	cost: 0.36614487 	acc: 0.7661499
1000 	cost: 0.35158038 	acc: 0.7667959
1200 	cost: 0.33678615 	acc: 0.77131784
1400 	cost: 0.33094656 	acc: 0.751938
1600 	cost: 0.31489202 	acc: 0.74547803
1800 	cost: 0.31130528 	acc: 0.748708
2000 	cost: 0.31111312 	acc: 0.7629199
2200 	cost: 0.30164725 	acc: 0.75775194
2400 	cost: 0.29535374 	acc: 0.77067184
2600 	cost: 0.2822594 	acc: 0.76937985
2800 	cost: 0.2714449 	acc: 0.75775194
3000 	cost: 0.27218685 	acc: 0.7629199
3200 	cost: 0.267765 	acc: 0.7648579
3400 	cost: 0.2640853 	acc: 0.7609819
3600 	cost: 0.26451153 	acc: 0.751292
3800 	cost: 0.26865396 	acc: 0.7655039
4000 	cost: 0.25838754 	acc: 0.7732558
4200 	cost: 0.25614098 	acc: 0.7751938
4400 	cost: 0.2649098 	acc: 0.76937985
4600 	cost: 0.2513209 	acc: 0.75775194
4800 	cost: 0.26363042 	acc: 0.7596899
5000 	cost: 0.2525931 	acc: 0.770